In [1]:
import pypyodbc
import sys
import pandas as pd 
import time
sys.path.append('../')
from KNNmodel import *
from rec_helper import *
from tqdm import tqdm
import scipy.sparse as sp          

In [2]:
con = pypyodbc.connect("DRIVER={SQL Server};SERVER=dbm_public;UID=sa;PWD=01060728;DATABASE=project2017")

In [3]:
df = pd.read_sql("select * from 基金推薦_近二年申購_憑證歸戶 ",con)

In [4]:
df.head()

,身分證字號,憑證,基金代碼,商品投資屬性,申購登錄日,扣款次數,申購扣款金額_台幣,國內外基金註記,aum基金型態別,投資型態,投資地區,型態別,aum型態別,aum計算類別
0,G2723186890,120J0W41600368,J0W,RR3,20160701,1,234180.8,1.0,B,b.單筆申購,a.境外基金,b.債券型,b.債券型,國外債券型
1,A1235025000,136317C1700029,317,RR3,20170811,1,496793.0,0.0,B,b.單筆申購,b.國內基金,b.債券型,b.債券型,國內債券型
2,F2778691990,18463Z51700045,63Z,RR5,20171011,3,13571.1,1.0,E,a.定時定額,a.境外基金,a.股票型,a.股票型,國外股票型
3,D2724924750,023Y3801700178,Y38,RR4,20171027,1,500000.0,1.0,E,b.單筆申購,a.境外基金,a.股票型,a.股票型,國外股票型
4,D2721254300,013J1F11600058,J1F,RR3,20160504,4,2000000.0,1.0,W,a.定時定額,a.境外基金,d.其他型,d.平衡型,國外其他型


In [5]:
# 建立u-i 矩陣 至少買過一檔基金
df_gt2 = threshold_interaction(df,rowname='身分證字號',colname='基金代碼',row_min=1,col_min=0) 
purchased_ui, userid_to_idx, \
    idx_to_userid, itemid_to_idx,idx_to_itemid  = df_to_spmatrix(df_gt2,'身分證字號','基金代碼')
train,test, user_idxs = train_test_split(purchased_ui,split_count=1,fraction=0.2)     

Starting interactions info
Number of rows: 35122
Number of cols: 2059
Sparsity: 0.20%
Ending interactions info
Number of rows: 19748
Number of columns: 2028
Sparsity: 0.32%


## KNN model 

In [26]:
t1 = time.time()
model_i = KNNmodel(train,kind='ibcf')
model_i.jaccard_sim()
model_i.fit(topK=100,remove=True)
t2 = time.time()
dt_ibcf = t2-t1
print('\n***'*10)
print('time cost for ibcf:{:.1f}s'.format(dt_ibcf))

similarity (jaccard) matrix built (ibcf), 
sparsity of similarity: 8.19 %


100%|█████████████████████████████████████████████████████| 2028/2028 [00:32<00:00, 63.03items/s]


ibcf rating matrix built...

handling nan data...


100%|███████████████████████████████████████████| 12389346/12389346 [00:10<00:00, 1227377.40it/s]


numbers of rows : 19748
numbers of cols: 2028
sparsity of rating: 30.94 %
save into *.rating attribute...

***
***
***
***
***
***
***
***
***
***
time cost for ibcf:53.6s


In [27]:
## ubcf
t1 = time.time()
model_u = KNNmodel(train,kind='ubcf')
model_u.jaccard_sim()
model_u.fit(topK=100,remove=True)
t2 = time.time()
dt_ubcf = t2-t1
print('\n***'*10)
print('time cost for ubcf:{:.1f}s'.format(dt_ubcf))

similarity (jaccard) matrix built (ubcf), 
sparsity of similarity: 11.21 %
-- start building topK user array...


100%|█████████████████████████████████████████████████| 19748/19748 [00:15<00:00, 1237.83users/s]


-- end building topK user array---
start building prediction rating...


100%|██████████████████████████████████████████████████| 19748/19748 [00:31<00:00, 621.65users/s]


ubcf rating matrix built...

handling nan data...


100%|█████████████████████████████████████████████| 2225816/2225816 [00:01<00:00, 1158099.60it/s]


numbers of rows : 19748
numbers of cols: 2028
sparsity of rating: 5.56 %
save into *.rating attribute...

***
***
***
***
***
***
***
***
***
***
time cost for ubcf:71.9s


In [28]:
## popular
t1 = time.time()
model_p = KNNmodel(train,kind='popular')
model_p.fit(topK=100,remove=True)
t2 = time.time()
dt_pop = t2-t1
print('\n***'*10)
print('time cost for popular:{:.1f}s'.format(dt_pop))

100%|████████████████████████████████████████████████████| 19748/19748 [00:03<00:00, 6256.54it/s]


popular rating matrix built...

handling nan data...


100%|█████████████████████████████████████████████| 1974800/1974800 [00:01<00:00, 1108613.50it/s]


numbers of rows : 19748
numbers of cols: 562
sparsity of rating: 17.79 %
save into *.rating attribute...

***
***
***
***
***
***
***
***
***
***
time cost for popular:5.9s


## 評估模型 , 
    - recall/ precision


In [29]:
uids = np.arange(0,train.shape[0])

predall_u = model_u.predict(uids,topN=10) # np array (itemidx)
model_u.evaluate(predall_u,test,method='recall') # 29.27

predall_i = model_i.predict(uids,topN=10) #nparray (itemidx)
model_i.evaluate(predall_i,test,method='recall') # 12.71 

predall_p = model_p.predict(uids,topN=10) #nparray (itemidx)
model_p.evaluate(predall_p,test,method='recall') # 20.08



-------------
model: ubcf,
topN: 10
recall:29.27 %

-------------
model: ibcf,
topN: 10
recall:12.71 %

-------------
model: popular,
topN: 10
recall:20.08 %


## 推薦清單 

In [39]:
fundid_names_df = pd.read_csv('../funds/fundid_to_name.csv',encoding='cp950')
fundid_to_names = {}

for d in fundid_names_df.to_dict('records'):
    fundid_to_names[d['基金代碼']] = d['基金中文名稱']
#%% 

In [68]:
def sample_recommendation_knn(data,model,user_idxs, print_output=True):
    ## user_ids : list of user ids ,eg: ['A1234567890','B12345xxxxxx',...]
    train = data['train']
    test = data['test']
    idx_to_itemid = data['idx_to_itemid']
    idx_to_userid = data['idx_to_userid']
    
    uidxs = np.array(user_idxs) # convert to uidxs np.array
    rec_fundidxs = model.predict(uidxs)
    for _, uidx in enumerate(user_idxs):
        known_positives_itemids = [ 
            idx_to_itemid[e] for e in train[uidx].indices
        ]
        known_positives_item_names = [
            fundid_to_names[fundid] for fundid in known_positives_itemids
        ]
        
        if print_output == True:
            print("User %s" % idx_to_userid[uidx])
            print("     ===Known positives(top3)===")

            for x in known_positives_item_names[:3]:
                print("        %s" % x)

            print("     ===Recommended(top3)===")

            for x in rec_fundidxs[_,:3]:
                print("        %s" % fundid_to_names[idx_to_itemid[x]])
            print('\n')

In [69]:
data = {'train': train, 'test':test, 'idx_to_itemid': idx_to_itemid, 'idx_to_userid':idx_to_userid}
sample_recommendation_knn(data,model_u,[4,10,20])

User D2721254300
     ===Known positives(top3)===
        (百元基金)摩根策略總報酬基金-JPM-A股累計(美元對沖)
        (百元基金)永豐滬深300紅利指數基金
        鋒裕新興市場債券基金TXD配息(美元)
     ===Recommended(top3)===
        路博邁投資基金-NB新興市場股票基金T累積類股(美元)
        (百元基金)貝萊德世界礦業基金(美元)
        (百元基金)摩根俄羅斯基金-JPM-A股分派(美元)


User N2221823430
     ===Known positives(top3)===
        (百元基金)摩根策略總報酬基金-JPM-A股累計(美元對沖)
        摩根亞太入息基金-JPM-A股每月派息(美元)
        摩根多重收益基金-JPM-A股每月派息(美元對沖)
     ===Recommended(top3)===
        安聯收益成長基金-AM(穩定月收類股)(美元)
        路博邁投資基金-NB新興市場股票基金T累積類股(美元)
        路博邁AR台灣股票基金-T月配(新臺幣)


User D2297379550
     ===Known positives(top3)===
        聯博全球高收益債券基金TA月配(人民幣)
        (百元基金)摩根策略總報酬基金-JPM-A股累計(美元對沖)
        (百元基金)摩根俄羅斯基金-JPM-A股分派(美元)
     ===Recommended(top3)===
        (百元基金)富蘭克林坦伯頓全球生技領航基金(美元)
        (百元基金)貝萊德世界黃金基金(美元)
        安聯收益成長基金-AM(穩定月收類股)(南非幣避險)




## implicit ALS

In [6]:
import implicit

In [7]:
# initialize a model
model_als = implicit.als.AlternatingLeastSquares(factors=50)

train_64 = train.astype('float64')
# train the model on a sparse matrix of item/user/confidence weights
model_als.fit(train_64.T) 

# recommend items for a user
# user_items = item_user_data.T.tocsr()
useridx = 10
recommendations = model_als.recommend(0, train_64)

# find related items
itemidx = 10 
related = model_als.similar_items(itemidx)

In [8]:
print('shape of user_factor:',model_als.user_factors.shape)
print('shape of item_factor:',model_als.item_factors.shape)


shape of user_factor: (19748, 50)
shape of item_factor: (2028, 50)


In [11]:
from sklearn.metrics import mean_squared_error
def calculate_mse(model, ratings, user_index=None):
    """Recommend products for all customers"""
    preds = model.user_factors.dot(model.item_factors.T)
#     preds = model.predict_for_customers()
    if user_index:
        return mean_squared_error(ratings[user_index, :].toarray().ravel(),
                                  preds[user_index, :].ravel())
    
    return mean_squared_error(ratings.toarray().ravel(),
                              preds.ravel())

In [36]:
def precision_at_k(model, ratings, k=5, user_index=None):
    if not user_index:
        user_index = range(ratings.shape[0])
    ratings = ratings.tocsr()
    precisions = []
    # Note: line below may become infeasible for large datasets.
#     predictions = model.predict_for_customers()
    predictions = model.user_factors.dot(model.item_factors.T)
    for user in user_index:
        # In case of large dataset, compute predictions row-by-row like below
        # predictions = np.array([model.predict(row, i) for i in xrange(ratings.shape[1])])
        top_k = np.argsort(-predictions[user, :])[:k]
        labels = ratings.getrow(user).indices
        if np.any(labels):
            precision = float(len(set(top_k) & set(labels))) / float(k)
            precisions.append(precision)
    return np.mean(precisions) 

In [35]:
if np.any(test.getrow(0).indices):
    print('h')

In [13]:
def print_log(row, header=False, spacing=12):
    top = ''
    middle = ''
    bottom = ''
    for r in row:
        top += '+{}'.format('-'*spacing)
        if isinstance(r, str):
            middle += '| {0:^{1}} '.format(r, spacing-2)
        elif isinstance(r, int):
            middle += '| {0:^{1}} '.format(r, spacing-2)
        elif isinstance(r, float):
            middle += '| {0:^{1}.5f} '.format(r, spacing-2)
        bottom += '+{}'.format('='*spacing)
    top += '+'
    middle += '|'
    bottom += '+'
    if header:
        print(top)
        print(middle)
        print(bottom)
    else:
        print(middle)
        print(top)

In [12]:

row2 = [1,2,3,4]
row3 = [1.32,1.22,3.231,4.211]
row = ['hello','my','name','is']
# print_log(row)
print_log(row, header=True, spacing=12)
print_log(row2, header=False,spacing=12)
print_log(row3)

+------------+------------+------------+------------+
|   hello    |     my     |    name    |     is     |
+============+============+============+============+
|     1      |     2      |     3      |     4      |
+------------+------------+------------+------------+
|  1.32000   |  1.22000   |  3.23100   |  4.21100   |
+------------+------------+------------+------------+


In [75]:
def learning_curve(train, test, epochs, num_factor=50, regularization =0 , k=5, user_index=None):
    if not user_index:
        user_index = range(train.shape[0])
    prev_epoch = 0
    train_precision = []    
    test_precision = []
    topK = []
    
    train = train.astype('float64')
    
    headers = ['epochs', 'p@k train', 'p@k test','topK']
    print_log(headers, header=True)
    
    for epoch in epochs:
        model = implicit.als.AlternatingLeastSquares(factors=num_factor,iterations=epoch,regularization=regularization)
        model.fit(train.T)

        train_precision.append(precision_at_k(model, train, k, user_index))
        test_precision.append(precision_at_k(model, test, k, user_index))
        topK.append(k)
        
        row = [epoch, train_precision[-1], test_precision[-1],k]
        
        print_log(row)
        prev_epoch = epoch
    return model, train_precision, test_precision, topK

In [40]:
learning_curve(train,test,epochs=[5,10,15,20],k=10)

+------------+------------+------------+------------+
|   epochs   | p@k train  |  p@k test  |    topK    |
+============+============+============+============+
|     5      |  0.26465   |  0.00975   |     10     |
+------------+------------+------------+------------+
|     10     |  0.26048   |  0.01046   |     10     |
+------------+------------+------------+------------+
|     15     |  0.25942   |  0.01024   |     10     |
+------------+------------+------------+------------+
|     20     |  0.25878   |  0.01024   |     10     |
+------------+------------+------------+------------+


(<implicit.als.AlternatingLeastSquares at 0x5476d07ba8>,
 [0.26464600425402612,
  0.26048313582497723,
  0.25942469360883219,
  0.25877646105540369],
 [0.0097542437294147466,
  0.010463643273372181,
  0.010235621991385864,
  0.010235621991385864],
 [10, 10, 10, 10])

In [65]:
import copy

In [76]:
def grid_search_learning_curve(train, test, param_grid,
                               user_index=None, patk=5, epochs=range(2, 40, 2)):
    """
    "Inspired" (stolen) from sklearn gridsearch
    https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_search.py
    """
    curves = []
    keys, values = zip(*param_grid.items())
    for v in itertools.product(*values):
        params = dict(zip(keys, v))
        
        print_line = []
        for k, v in params.items():            
            print_line.append((k, v))

        print(' | '.join('{}: {}'.format(k, v) for (k, v) in print_line))
        _, train_patk, test_patk, topK = learning_curve(train, test, epochs, params.get('factors'), params.get('regularization'),
                                                        patk, user_index=user_index)
        curves.append({'params': params,
                       'patk': {'train': train_patk, 'test': test_patk},
                       })
    return curves

In [82]:
param_grid = {'factors': [5,10,20,30,40,50],
              'regularization': [0.0, 1e-5, 1e-3, 1e-1,1e0, 1e1, 1e2]}

In [83]:
%time eva = grid_search_learning_curve(train,test,param_grid,patk=10,epochs = range(10,41,10))

factors: 5 | regularization: 0.0
+------------+------------+------------+------------+
|   epochs   | p@k train  |  p@k test  |    topK    |
+============+============+============+============+
|     10     |  0.12819   |  0.02148   |     10     |
+------------+------------+------------+------------+
|     20     |  0.13201   |  0.02214   |     10     |
+------------+------------+------------+------------+
|     30     |  0.13000   |  0.02186   |     10     |
+------------+------------+------------+------------+
|     40     |  0.13068   |  0.02194   |     10     |
+------------+------------+------------+------------+
factors: 5 | regularization: 1e-05
+------------+------------+------------+------------+
|   epochs   | p@k train  |  p@k test  |    topK    |
+============+============+============+============+
|     10     |  0.13021   |  0.02181   |     10     |
+------------+------------+------------+------------+
|     20     |  0.13210   |  0.02247   |     10     |
+------------+

## Learning to Rank
- lightFM